<a href="https://colab.research.google.com/github/drakeai-96/hcc/blob/main/karaciger_calismasi_github.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Karaciğer Kanseri (HCC) Nüks Riskini Hesaplama Projesi
# Veri Ön İşleme ve Gradient Boosting Modeli ile Tahmin

# Gerekli Kütüphaneler
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from datetime import datetime

# Veri Yükleme
veriler = pd.read_csv("tum_veriler.csv")

# Kullanılacak parametrelerin seçimi
parametreler = [
    'HASTA No', 'Ad Soyad', 'Cinsiyet \nErkek: 1\nKadın:2', 'Doğum Tarihi\nGün Ay Yıl\n00.00.0000 ',
    'Boy \ncm', 'Kilo\nkg', 'Albumin (gr/dL)', 'INR\nDüz.', 'Kreatinin', 'total Bil.',
    'Plt', 'AST', 'ALT', 'ALP', 'GGT', 'Hb', 'AFP', 'MR/CT de \nAna Tümör Boyutu',
    'MR/CT de \nTümör Sayısı', 'Ameliyat Tarihi\nGün Ay Yıl\n00.00.0000 ',
    'Nüks Durum\n0:0\nVar:1', 'Nüks Tarihi\nGün Ay Yıl\n00.00.0000 \n'
]

veriler = veriler[parametreler]

# Veri Temizleme
sayisal_sutunlar = ['Boy \ncm', 'Kilo\nkg', 'Albumin (gr/dL)', 'INR\nDüz.', 'Kreatinin',
                   'total Bil.', 'Plt', 'AST', 'ALT', 'ALP', 'GGT', 'Hb', 'AFP',
                   'MR/CT de \nAna Tümör Boyutu', 'MR/CT de \nTümör Sayısı']

for sutun in sayisal_sutunlar:
    veriler[sutun] = (
        veriler[sutun].astype(str)
        .str.replace("[<->]", "0", regex=True)
        .str.replace(",", ".", regex=True)
        .str.extract('(\d+\.?\d*)')[0]
        .astype(float)
    )

# Cinsiyet verisini numerik hale getirme
veriler['Cinsiyet'] = veriler['Cinsiyet \nErkek: 1\nKadın:2'].replace({'Erkek': 1, 'Kadın': 2}).astype(float)
veriler['Erkek'] = (veriler['Cinsiyet'] == 1).astype(int)
veriler['Kadın'] = (veriler['Cinsiyet'] == 2).astype(int)
veriler.drop('Cinsiyet \nErkek: 1\nKadın:2', axis=1, inplace=True)

# Tarihleri datetime formatına çevirme
for tarih_sutunu in ['Doğum Tarihi\nGün Ay Yıl\n00.00.0000 ', 'Ameliyat Tarihi\nGün Ay Yıl\n00.00.0000 ',
                     'Nüks Tarihi\nGün Ay Yıl\n00.00.0000 \n']:
    veriler[tarih_sutunu] = pd.to_datetime(veriler[tarih_sutunu], errors='coerce', dayfirst=True)

# Eksik verilerin KNN ile doldurulması
imputer = KNNImputer(n_neighbors=5)
veriler[sayisal_sutunlar] = imputer.fit_transform(veriler[sayisal_sutunlar])

# Yaş hesaplama
bugun = datetime.today()
veriler['Yas'] = veriler['Doğum Tarihi\nGün Ay Yıl\n00.00.0000 '].apply(lambda x: bugun.year - x.year if pd.notnull(x) else np.nan)

# Hedef sütunu belirleme ve gereksiz sütunları kaldırma
hedef = 'Nüks Durum\n0:0\nVar:1'
veriler_final = veriler.drop(['Doğum Tarihi\nGün Ay Yıl\n00.00.0000 ',
                              'Ameliyat Tarihi\nGün Ay Yıl\n00.00.0000 ',
                              'Nüks Tarihi\nGün Ay Yıl\n00.00.0000 \n', 'Ad Soyad'], axis=1)

# Eğitim ve Test setlerine bölme
X = veriler_final.drop(hedef, axis=1)
y = veriler_final[hedef]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Gradient Boosting Modeli oluşturma ve eğitme
model = GradientBoostingClassifier()
model.fit(X_train, y_train)

# Tahmin ve Değerlendirme
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:, 1]

accuracy = accuracy_score(y_test, y_pred)
print(f"Model doğruluğu (Accuracy): {accuracy:.2f}")
print(classification_report(y_test, y_pred))

In [ ]:
# Karaciğer Kanseri (HCC) Nüks Riskini Hesaplama Projesi
# 30 Kişilik Örnek Dataset Üzerinde Gradient Boosting Modeli

# Gerekli Kütüphaneler
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Veri Yükleme
veriler = pd.read_csv("karaciger_dataset_30_kisi_corrected.csv")

# Gereksiz sütunları çıkarma
veriler.drop(['HASTA No', 'Ad Soyad', 'Doğum Tarihi\nGün Ay Yıl\n00.00.0000 ',
              'Ameliyat Tarihi\nGün Ay Yıl\n00.00.0000 ', 'Nüks Tarihi\nGün Ay Yıl\n00.00.0000 \n'],
             axis=1, inplace=True)

# Bağımlı ve bağımsız değişkenleri ayırma
X = veriler.drop('Nüks Durum\n0:0\nVar:1', axis=1)
y = veriler['Nüks Durum\n0:0\nVar:1']

# Eğitim ve Test setlerine bölme
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Gradient Boosting Modeli oluşturma ve eğitme
model = GradientBoostingClassifier()
model.fit(X_train, y_train)

# Tahmin ve Değerlendirme
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:, 1]

accuracy = accuracy_score(y_test, y_pred)
print(f"Model doğruluğu (Accuracy): {accuracy:.2f}")
print(classification_report(y_test, y_pred))

Model doğruluğu (Accuracy): 0.00
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       5.0
           1       0.00      0.00      0.00       1.0

    accuracy                           0.00       6.0
   macro avg       0.00      0.00      0.00       6.0
weighted avg       0.00      0.00      0.00       6.0

